# Importing required libraries

In [ ]:
!pip install 'tensorflow==1.15.3'

In [ ]:
import matplotlib.pyplot as plt # plotting
from PIL import Image
import matplotlib.patches as patches
import matplotlib.image as IMG
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.io
import tensorflow as tf
import tensorflow_addons as tfa


# **How to read mat files?**

In [ ]:
def mat_to_boundbox(filename):
    input = scipy.io.loadmat(filename)['boxes']
    box_numbers = input.shape[-1]
    bx1_e1 = input[0][0][0][0][0][0]
    bx1_e2 = input[0][0][0][0][1][0]
    bx1_e3 = input[0][0][0][0][2][0]
    bx1_e4 = input[0][0][0][0][3][0]
    bx1 = np.array([bx1_e1,bx1_e2,bx1_e3,bx1_e4])
    output = np.array(bx1)
    output = np.flip(output,1)
    output = np.reshape(output,(1,4,2))
    if box_numbers == 2:
        bx2_e1 = input[0][1][0][0][0][0]
        bx2_e2 = input[0][1][0][0][1][0]
        bx2_e3 = input[0][1][0][0][2][0]
        bx2_e4 = input[0][1][0][0][3][0]
        bx2 = [bx2_e1,bx2_e2,bx2_e3,bx2_e4]
        output = np.array([bx1,bx2])
        output = np.flip(output,2)
   
    return output
    

# Draw bounding box

In [ ]:
def drawBoundbox(image,coordinate):
    plt.figure()
    plt.imshow(img)
    if coordinate.shape[0] >= 1:
        plt.scatter(x=[coordinate[0][0][0],coordinate[0][1][0],coordinate[0][2][0],coordinate[0][3][0]], y=[coordinate[0][0][1],coordinate[0][1][1],coordinate[0][2][1],coordinate[0][3][1]], c='r', s=20)
    if coordinate.shape[0] == 2:
        plt.scatter(x=[coordinate[1][0][0],coordinate[1][1][0],coordinate[1][2][0],coordinate[1][3][0]], y=[coordinate[1][0][1],coordinate[1][1][1],coordinate[1][2][1],coordinate[1][3][1]], c='g', s=20)
    plt.show()

# Resize and edit bounding boxs coordinates

In [ ]:
def resize_edit_box(imagefile,coordinatefile):
    img = Image.open(imagefile)
    img1 = img.resize((512,512),0)
    img_shape = np.array(img).shape
    img = np.array(img1)
    
    x_scale = 512 / img_shape[1]
    y_scale = 512/ img_shape[0]
    
    coordinate = mat_to_boundbox(coordinatefile)
    
    if coordinate.shape[0] == 1:
        changed_x_coordinate = np.reshape(coordinate[:,:,0]*x_scale,(1,4,1))
        changed_y_coordinate = np.reshape(coordinate[:,:,1]*y_scale,(1,4,1))
    if coordinate.shape[0] == 2:
        changed_x_coordinate = np.reshape(coordinate[:,:,0]*x_scale,(2,4,1))
        changed_y_coordinate = np.reshape(coordinate[:,:,1]*y_scale,(2,4,1))
        
    coordinate = np.concatenate((changed_x_coordinate, changed_y_coordinate), axis=2)
    
    return img , coordinate


In [ ]:
testImage_path = '/kaggle/input/hand_dataset/training_dataset/training_data/images/Buffy_110.jpg'
testAnnotation_path = '/kaggle/input/hand_dataset/training_dataset/training_data/annotations/Buffy_110.mat'

img, coordinate = resize_edit_box(testImage_path,testAnnotation_path)
drawBoundbox(img, coordinate)


# Chenging Coordinate format to Center(X),Center(Y),Height , Weight

In [ ]:
def distance(point1,point2):
    distance = ((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)**0.5
    return distance



def coordinate_reformat(coordinate):
    if coordinate.shape[0] >= 1:
        x_center = (coordinate[0,0,0] + coordinate[0,2,0])/2
        y_center = (coordinate[0,0,1] + coordinate[0,2,1])/2
        height = np.amax(coordinate[0,:,1]) - np.amin(coordinate[0,:,1])
        weight = np.amax(coordinate[0,:,0]) - np.amin(coordinate[0,:,0])
        output = np.array([x_center,y_center,height,weight]).reshape(1,4,1)
        
        
    if coordinate.shape[0] == 2:
        x_center_2 = (coordinate[1,0,0] + coordinate[1,2,0])/2
        y_center_2 = (coordinate[1,0,1] + coordinate[1,2,1])/2
        height_2 = np.amax(coordinate[1,:,1]) - np.amin(coordinate[1,:,1])
        weight_2 = np.amax(coordinate[1,:,0]) - np.amin(coordinate[1,:,0])
        temp = np.array([x_center_2,y_center_2,height_2,weight_2]).reshape(1,4,1)
        output = np.concatenate((output, temp), axis=0).reshape(2,4,1)
        
    return output


def coordinate_main(Newcoordinate):
    center_x = Newcoordinate[0,0,0]
    center_y = Newcoordinate[0,1,0]
    height = Newcoordinate[0,2,0]
    weidth = Newcoordinate[0,3,0]
    
    if Newcoordinate.shape[0] >= 1:
        a = np.array([center_x - (height/2 + weidth/2), center_y - (height/2 + weidth/2)])
        b = np.array([center_x - (height/2 - weidth/2), center_y - (height/2 - weidth/2)])
        c = np.array([center_x - (-height/2 - weidth/2), center_y - (-height/2 - weidth/2)])
        d = np.array([center_x - (-height/2 + weidth/2), center_y - (-height/2 + weidth/2)])
        temp1 = np.concatenate((a,b,c,d),axis = 0).reshape(1,4,2)
        
    
    if Newcoordinate.shape[0] == 2:
        center_x = Newcoordinate[1,0,0]
        center_y = Newcoordinate[1,1,0]
        height = Newcoordinate[1,2,0]
        weidth = Newcoordinate[1,3,0]
        a = np.array([center_x - (height/2 + weidth/2), center_y - (height/2 + weidth/2)])
        b = np.array([center_x - (height/2 - weidth/2), center_y - (height/2 - weidth/2)])
        c = np.array([center_x - (-height/2 - weidth/2), center_y - (-height/2 - weidth/2)])
        d = np.array([center_x - (-height/2 + weidth/2), center_y - (-height/2 + weidth/2)])
        temp2 = output = np.concatenate((a,b,c,d),axis = 0).reshape(1,4,2)
        
        output = np.concatenate((temp1, temp2), axis = 0)
        
    return output

# Find appropriate anchor boxes

In [ ]:
all_coordinate = np.empty([2])
for dirname, _, filenames in os.walk('../input/hand_dataset/training_dataset/training_data/annotations'):
    for filename in filenames:
        if 'Buffy' in filename: 
            temp = coordinate_reformat(mat_to_boundbox(os.path.join(dirname, filename))).reshape(1,-1,4)
            if all_coordinate.shape[0] == 2:
                all_coordinate = temp
            else:
                all_coordinate = np.concatenate((temp,all_coordinate),axis = 1)
                
print(all_coordinate.shape)
plt.plot(all_coordinate[0,:,2],all_coordinate[0,:,3],'ro') 
plt.show()

In [ ]:
from sklearn.cluster import KMeans
all_coordinate = all_coordinate.reshape(-1,4)
HW = all_coordinate[:,2:4]
kmeans = KMeans(n_clusters=1, random_state=0).fit(HW)
kmeans.cluster_centers_

**So appropriate anchor size is [51.49931633, 49.05936578]**

In [ ]:
anchor_DEFUALT = np.array([51.49931633, 49.05936578])

# Creating CSV dataset

In [ ]:
def create_output(imgFile, annoFile, grid = 16):
    img, coordinate = resize_edit_box(imgFile, annoFile)
    coordinate = coordinate_reformat(coordinate)
    step = img.shape[1]/grid
    output = np.zeros((5,grid,grid))
    boundBox_grid_x = int(coordinate[0,0,0]/step)
    if boundBox_grid_x == 16: boundBox_grid_x = 15
    boundBox_grid_y = int(coordinate[0,1,0]/step)
    if boundBox_grid_y == 16: boundBox_grid_y = 15
    boundBox_x = (coordinate[0,0,0]% step)/step
    boundBox_y = (coordinate[0,1,0]% step)/step
    boundBox_height = coordinate[0,2,0]/(anchor_DEFUALT[0] * grid)
    boundBox_weidth = coordinate[0,3,0]/(anchor_DEFUALT[1] * grid)
    temp = np.array([1, boundBox_x, boundBox_y, boundBox_height, boundBox_weidth]).reshape(5)
    output[:,boundBox_grid_x,boundBox_grid_y] = temp
    
    if coordinate.shape[0] == 2:
        boundBox_grid_x = int(coordinate[1,0,0]/step)
        if boundBox_grid_x == 16: boundBox_grid_x = 15
        boundBox_grid_y = int(coordinate[1,1,0]/step)
        if boundBox_grid_y == 16: boundBox_grid_y = 15
        boundBox_x = (coordinate[1,0,0]% step)/step
        boundBox_y = (coordinate[1,1,0]% step)/step
        boundBox_height = coordinate[1,2,0]/(anchor_DEFUALT[0] * grid)
        boundBox_weidth = coordinate[1,3,0]/(anchor_DEFUALT[1] * grid)
        temp1 = np.array([1, boundBox_x, boundBox_y, boundBox_height, boundBox_weidth]).reshape(5)
        output[:,boundBox_grid_x,boundBox_grid_y] = temp1
        
    return img, output
    
    

In [ ]:
import csv

csv_head = ["no."]
for i in range(0,262144):
    csv_head.append("pixel " + str(i))
for i in range(0,256):
    csv_head.append("Objectness score(gride"+str(i)+")"), csv_head.append("Center x(gride"+str(i)+")"), csv_head.append("Center y(gride"+str(i)+")"), csv_head.append("Height(gride"+str(i)+")"), csv_head.append("Width(gride"+str(i)+")")

In [ ]:
def create_dataset(images,annotations):
    counter = 0
    final_output = []
    temp = []
    for IMGdir, _, IMGfiles in os.walk(images):
        for ANNOdir, _, ANNOfiles in os.walk(annotations):
             for img_file in IMGfiles:
                    for anno_file in ANNOfiles:
                        if counter == 1:break
                        idx = img_file.index('.jpg')
                        if 'Buffy' in img_file and img_file[0:idx]+'.mat' == anno_file:
                            img = os.path.join(IMGdir, img_file)
                            ann = os.path.join(ANNOdir, anno_file)
                            img, out = create_output(img,ann)
                            img = np.reshape(img,(1,-1)).tolist()[0]
                            out = np.reshape(out,(1,-1),order='F').tolist()[0]
                            temp = [counter] + img + out
                            final_output.append(temp)
                            counter+=1
    return final_output

In [ ]:
trainimg_dir = '../input/hand_dataset/training_dataset/training_data/images'
trainanno_dir = '../input/hand_dataset/training_dataset/training_data/annotations'
#train = create_dataset(trainimg_dir, trainanno_dir)
testimg_dir = '../input/hand_dataset/test_dataset/test_data/images'
testanno_dir = '../input/hand_dataset/test_dataset/test_data/annotations'
#test = create_dataset(testimg_dir, testanno_dir)


In [ ]:
write = []
write.append(csv_head),write.append(train[0])
with open('dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(write)

In [ ]:
import pandas as pd
df = pd.read_csv("./dataset.csv")


# Data Generator

In [ ]:
def generator(images, annotations , batch_size=32):
    """
    Yields the next training batch.
    """
    
   
    while True:
        counter = 0
        X_train = []
        y_train = []
        for IMGdir, _, IMGfiles in os.walk(images):
            for ANNOdir, _, ANNOfiles in os.walk(annotations):
                 for img_file in IMGfiles:
                        for anno_file in ANNOfiles:
                            if counter == batch_size: 
                                X_train = np.array(X_train)
                                y_train = np.array(y_train)
                                yield X_train, y_train
                                X_train = []
                                y_train = []
                                counter = 0
                            idx = img_file.index('.jpg')
                            if 'Buffy' in img_file and img_file[0:idx]+'.mat' == anno_file:
                                img = os.path.join(IMGdir, img_file)
                                ann = os.path.join(ANNOdir, anno_file)
                                img, out = create_output(img,ann)
                                X_train.append(img)
                                y_train.append(out)
                                counter+=1

In [ ]:
train_generator = generator(trainimg_dir, trainanno_dir, batch_size=32)
test_generator =  generator(testimg_dir, testanno_dir, batch_size=32)

In [ ]:
x, y = next(train_generator)
print(x.shape)

# Model architecture

batch normalization and fixed padding

In [ ]:
_BATCH_NORM_DECAY = 0.9
_BATCH_NORM_EPSILON = 1e-05
_LEAKY_RELU = 0.1

In [ ]:
def batch_norm(inputs, training, data_format):
    """Performs a batch normalization using a standard set of parameters."""
    return tf.layers.batch_normalization(
        inputs=inputs, axis=1 if data_format == 'channels_first' else 3,
        momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON,
        scale=True, training=training)


def fixed_padding(inputs, kernel_size, data_format):
    """ResNet implementation of fixed padding.

    Pads the input along the spatial dimensions independently of input size.

    Args:
        inputs: Tensor input to be padded.
        kernel_size: The kernel to be used in the conv2d or max_pool2d.
        data_format: The input format.
    Returns:
        A tensor with the same format as the input.
    """
    pad_total = kernel_size - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg

    if data_format == 'channels_first':
        padded_inputs = tf.pad(inputs, [[0, 0], [0, 0],
                                        [pad_beg, pad_end],
                                        [pad_beg, pad_end]])
    else:
        padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                        [pad_beg, pad_end], [0, 0]])
    return padded_inputs


def conv2d_fixed_padding(inputs, filters, kernel_size, data_format, strides=1):
    """Strided 2-D convolution with explicit padding."""
    if strides > 1:
        inputs = fixed_padding(inputs, kernel_size, data_format)

    return tf.layers.conv2d(
        inputs=inputs, filters=filters, kernel_size=kernel_size,
        strides=strides, padding=('SAME' if strides == 1 else 'VALID'),
        use_bias=False, data_format=data_format)

Darknet 53

In [ ]:
def darknet53_residual_block(inputs, filters, training, data_format, strides=1):
    """Creates a residual block for Darknet."""
    shortcut = inputs

    inputs = conv2d_fixed_padding(
        inputs, filters=filters, kernel_size=1, strides=strides,
        data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(
        inputs, filters=2 * filters, kernel_size=3, strides=strides,
        data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs += shortcut

    return inputs


def darknet53(inputs, training, data_format):
    """Creates Darknet53 model for feature extraction."""
    inputs = conv2d_fixed_padding(inputs, filters=32, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    inputs = conv2d_fixed_padding(inputs, filters=64, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = darknet53_residual_block(inputs, filters=32, training=training,
                                      data_format=data_format)

    inputs = conv2d_fixed_padding(inputs, filters=128, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(2):
        inputs = darknet53_residual_block(inputs, filters=64,
                                          training=training,
                                          data_format=data_format)

    inputs = conv2d_fixed_padding(inputs, filters=256, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(8):
        inputs = darknet53_residual_block(inputs, filters=128,
                                          training=training,
                                          data_format=data_format)

    route1 = inputs

    inputs = conv2d_fixed_padding(inputs, filters=512, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(8):
        inputs = darknet53_residual_block(inputs, filters=256,
                                          training=training,
                                          data_format=data_format)

    route2 = inputs

    inputs = conv2d_fixed_padding(inputs, filters=1024, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(4):
        inputs = darknet53_residual_block(inputs, filters=512,
                                          training=training,
                                          data_format=data_format)

    return route1, route2, inputs

Yolo convolution

In [ ]:
def yolo_convolution_block(inputs, filters, training, data_format):
    """Creates convolution operations layer used after Darknet."""
    inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=2 * filters, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=2 * filters, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    route = inputs

    inputs = conv2d_fixed_padding(inputs, filters=2 * filters, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    return route, inputs

Detection layer

In [ ]:
def yolo_layer(inputs,data_format):
    
    n_anchors = 1
    n_classes = 0

    inputs = tf.layers.conv2d(inputs, filters=n_anchors * (5 + n_classes),
                              kernel_size=1, strides=1, use_bias=True,
                              data_format=data_format)

   
    if data_format != 'channels_first':
        inputs = tf.transpose(inputs, [0, 3,1,2])
    
    confidence, box_centers, box_shapes = \
                                        tf.split(inputs, [1, 2, 2], axis=1)

    confidence = tf.nn.sigmoid(confidence)
    box_centers = tf.nn.sigmoid(box_centers)
    box_shapes = tf.nn.leaky_relu(box_shapes)
    box_shape_height = box_shapes[:,0,:,:]
    box_shape_height = tf.reshape(box_shape_height,[1,1,16,16])
    box_shape_width = box_shapes[:,1,:,:]
    box_shape_width = tf.reshape(box_shape_width,[1,1,16,16])
    box_shape_height = box_shape_height * 512 * anchor_DEFUALT[0]
    box_shape_width = box_shape_width * 512 * anchor_DEFUALT[1]
    box_shapes = tf.concat([box_shape_height,box_shape_width], axis =1)
    inputs = tf.concat([confidence, box_centers, box_shapes], axis=1)

    return inputs

testing outut tensor shape

In [ ]:
inputs = tf.random.uniform(shape=[1,512,512,3])
print(inputs.get_shape(),"\n")
route1,route2, inputs = darknet53(inputs, training=True,data_format='channels_last')
print(inputs.get_shape(),"\n")
route, inputs = yolo_convolution_block(inputs, filters=512, training=True,data_format='channels_last')
print(inputs.get_shape(),"\n")
inputs = yolo_layer(inputs,data_format='channels_last')
print(inputs.get_shape(),"\n")

# Final YOLO model

In [ ]:
def ME_yolo(input_shape = (512,512,3),training=False, data_format='channels_last'):
    inputs = Input(shape=input_shape)
    route1,route2, inputs = darknet53(inputs,train, data_format)
    route, inputs = yolo_convolution_block(inputs,512,train,data_format)
    out = yolo_layer(inputs,data_format)
    model = Model(inputs=inputs,outputs=out)
    return model

# Customized Loss function

In [ ]:
def hw_to_minmax(tensor):
    
    batch_size = np.array(tensor.get_shape())[0]
    output = tf.zeros([batch_size,4,16,16])
    y_min = tensor[:,1,:,:] - (tensor[:,2,:,:]/2) - (tensor[:,3,:,:]/2)
    x_min = tensor[:,0,:,:] - (tensor[:,2,:,:]/2) - (tensor[:,3,:,:]/2)
    y_max = tensor[:,1,:,:] + (tensor[:,2,:,:]/2) + (tensor[:,3,:,:]/2)
    x_max = tensor[:,0,:,:] + (tensor[:,2,:,:]/2) + (tensor[:,3,:,:]/2)
    output = tf.concat([y_min, x_min, y_max, x_max],1)
    output = tf.reshape(output,[-1,4])
    return output

In [ ]:
temp1 = tf.reshape(tf.constant([5.0, 2.0, 6.0, 3.0]),[1,4,1,1])
temp2 = tf.reshape(tf.constant([5.0, 2.0, 6.0, 3.0]),[1,4,1,1])
temp = tf.concat([temp1,temp],0)

hw_to_minmax(temp).numpy()

In [ ]:
gl = tfa.losses.GIoULoss()
boxes1 = tf.constant([5.0, 2.0, 6.0, 3.0])
boxes2 = tf.constant([5.0, 2.0, 8.0, 3.0])
loss = gl(boxes1, boxes2)
print('Loss: ', loss.numpy())  # Loss: [1.07500000298023224, 1.9333333373069763]

In [ ]:
gl = tfa.losses.GIoULoss()
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    y_pred = model(x, training=training)
    confidence_true = y[:,0,:,:]
    confidence_pred = y_pred[:,0,:,:]
    loss_confidence = tf.keras.losses.MSE(confidence_true, confidence_pred)
    boundbox_true = hw_to_minmax(y[:,1:4,:,:])
    boundbox_pred = hw_to_minmax(y_pred[:,1:4,:,:])
    loss_boundbox = gl(boundbox_true, boundbox_pred)
    total_loss = tf.reduce_sum(loss_confidence) + tf.reduce_sum(loss_boundbox)
    
    return total_loss